In [56]:
import pandas as pd
import json
import collections
import numpy as np

In [4]:
def accuracy(truth_json, predicted_path):
    
    predicted = pd.read_csv(predicted_path)
    correct = 0
    truth_dict = collections.Counter()
    
    with open(truth_json, 'r') as json_file:
        json_list = list(json_file)

    for i, json_str in enumerate(json_list):
        result = json.loads(json_str)
        truth_dict[result['label']] += 1
        if result['label'] == predicted['label'][i]:
            correct += 1
    print(correct, len(json_list))
    print(truth_dict.items())

    return


In [57]:
def json_to_csv(truth_json):
    dict = collections.defaultdict(list)
    with open(truth_json, 'r') as json_file:
        json_list = list(json_file)
    for i, json_str in enumerate(json_list):
        result = json.loads(json_str)
        dict['label'].append(result['label'])
        dict['id'].append(result['id'])
        dict['text'].append(result['text'])
    df = pd.DataFrame(data=dict)
    return df

def log_to_proba(file):
    df = pd.read_csv(file)
    df['proba'] = df['proba'].apply(lambda x: np.exp(x))
    
    df.to_csv(file)
    return df
        
        
    

In [60]:
log_to_proba("./vilio/Ensemble_files/U36_test_seen.csv")
log_to_proba("./vilio/data/O36/O36_test_seen_SA.csv")

,id,proba,label
0,16395,0.969714,1
1,37405,0.031993,1
2,94180,0.030658,0
3,54321,0.339809,0
4,97015,0.995357,1
...,...,...,...
995,3869,0.000435,0
996,23817,0.002383,0
997,56280,0.186905,0
998,29384,0.000658,0


In [61]:
test_ground_truth = json_to_csv('./vilio/data/test_unseen.jsonl')
test_ground_truth.to_csv('./vilio/Ensemble_files/test.csv')

In [5]:
accuracy("./vilio/data/test_unseen.jsonl", "./vilio/data/O36/O36_test_seen_SA.csv")

589 1000
dict_items([(0, 753), (1, 247)])


In [66]:
class Ensemble:
    def __init__(self, ground_truth, uniter, oscar, visual_bert=None, vilbert=None):
        self.ground_truth = pd.read_csv(ground_truth)
        self.uniter = pd.read_csv(uniter)
        self.oscar = pd.read_csv(oscar)
        self.visual_bert = visual_bert
        self.vilbert = vilbert
    
    def compare(self, predict):
        correct = 0
        all = len(predict)
        for i in range(all):
            if predict['result'][i] == self.ground_truth['label'][i]:
                correct += 1
        print(correct, '/', all, "Accuracy:", correct/all)
        
    
    def simple_average(self):
        
        combined = pd.merge(self.uniter, self.oscar, on='id', suffixes=('_u', '_x'))
        
        cols = ['proba_u', 'proba_x']
        
        combined['avg'] = combined[cols].mean(axis=1)
        combined['result'] = combined['avg'].apply(lambda x: 1 if x >= 0.5 else 0)
        self.compare(combined)
        
        
    
    def weighted_average(self):
        
        return 
        
        
        

In [67]:
ensemble = Ensemble("./vilio/Ensemble_files/test.csv", "./vilio/Ensemble_files/U36_test_seen.csv", "./vilio/data/O36/O36_test_seen_SA.csv", None, None)
ensemble.simple_average()

597 / 1000 Accuracy: 0.597


In [8]:
!pwd

/home/jupyter/pdlsp-final-project


In [9]:
data = pd.read_csv("./vilio/Ensemble_files/U36_test_seen.csv")

        id     proba  label
0    16395 -0.000083      1
1    37405 -4.863254      0
2    94180 -4.268325      0
3    54321 -0.637010      1
4    97015 -0.000358      1
..     ...       ...    ...
995   3869 -8.740327      0
996  23817 -7.594582      0
997  56280 -0.003411      1
998  29384 -8.667371      0
999  34127 -0.130345      1

[1000 rows x 3 columns]
